In [ ]:
from pathlib import Path

import SAGA
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from easyquery import *
from SAGA.utils import *
from SAGA.utils.display import show_images
from SAGA.utils.distance import *
from astropy.table import vstack, Table, join
from astropy.coordinates import Distance, SkyCoord
from SAGA.objects.build2 import add_surface_brightness

import numexpr as ne
from scipy.stats import binned_statistic, norm
#from bottleneck import nanmax, nanmin
from itertools import product, chain

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = Query(C.basic_cut2).filter(base)
base = annotate_base(base)

In [ ]:
base_other = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_other.fits").read()
base_other = Query(C.basic_cut2).filter(base_other)
base_other = annotate_base(base_other)

In [ ]:
p1_hosts = saga.host_catalog.load(include_stats=True, query="paper1_complete")["HOSTID"]
p2_hosts = saga.host_catalog.load(include_stats=True, query="paper2_complete")["HOSTID"]

In [ ]:
rbins = np.linspace(-20.25, -11.75, 18)
#rbins = np.linspace(16.75, 21, )
r = 0.5*(rbins[1:]+rbins[:-1])

mag_label = "Mr_"

hostid = []
X = []
for base_this in chain(base.group_by("HOSTID").groups, base_other.group_by("HOSTID").groups):
    hostid.append(base_this["HOSTID"][0])
    for q in (
        C.paper2_targeting_cut,
        C.paper2_targeting_cut & C.has_spec,
        C.paper1_targeting_cut,
        C.paper1_targeting_cut & C.has_spec,
        C.paper2_targeting_cut & (~C.has_spec) & "human_selected > 0",
        C.paper1_targeting_cut & (~C.has_spec) & "human_selected > 0",
        C.paper2_targeting_cut & C.has_spec & (~C.has_our_specs_only),
    ):
        X.extend(np.histogram(q.filter(base_this, mag_label), rbins)[0])
X = np.array(X, dtype=np.float64).reshape(len(hostid), -1, len(r))
hostid = np.array(hostid)
print(X.shape)

with np.errstate(invalid="ignore"):
    X_ratio = np.nan_to_num(X[:,1] / X[:,0], nan=1)
    X_ratio_pre = np.nan_to_num(X[:,6] / X[:,0], nan=1)
    X_sum_ratio = X[:,1].sum(axis=1) / X[:,0].sum(axis=1) 
    X_sum = X[:,0].sum(axis=1) - X[:,1].sum(axis=1)
    X_sum_lsbg = X[:,5].sum(axis=1) - X[:,4].sum(axis=1)

In [ ]:
p1_mask = np.in1d(hostid, p1_hosts)
p2_mask = np.in1d(hostid, p2_hosts)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 3.7))

ax_this = ax[0]

for q, color, label, marker in (
    (~(p1_mask | p2_mask), 'C0', "Incomplete systems ({})", 'o'),
    (p2_mask & (~p1_mask), 'C3', 'New complete systems ({})', 's'),
    (p1_mask, 'C1', 'Paper I complete systems ({})', '^'),
):
    n = np.count_nonzero(q)
    ax_this.errorbar(X_sum_ratio[q], X_sum[q]*10**(np.random.randn(n)*0.01), 
                     marker=marker, ls='', label=label.format(n), ms=3.3, color=color, 
                    # yerr=(np.zeros(n), X_sum_lsbg[q]),
                    )
ax_this.set_yscale("log")
ax_this.legend(handletextpad=0.1, fontsize=13, frameon=True)
ax_this.set_ylim(1, 500)
ax_this.axvline(0.8, color="grey", ls="--")
ax_this.set_ylabel('# objects w/o redshift per host')
ax_this.set_xlabel('Spec coverage in primary targeting region')

ax_this = ax[1]
Y0 = np.mean(X_ratio[p2_mask], axis=0)
Yerr = np.percentile(X_ratio[p2_mask], norm.sf([1, -1])*100, axis=0)

ax_this.errorbar(r, Y0, (Y0-Yerr[0], Yerr[1]-Y0), ls=":", marker='o', color="C2", label="Spec coverage", zorder=8)
ax_this.errorbar(r, np.mean(X_ratio_pre, axis=0), ls="-", marker='', color="C8", label="Spec coverage\nbefore SAGA")

ax_this.set_ylim(0, 1.05)
ax_this.set_xlabel('$M_{r,o}$')
ax_this.set_ylabel('Spec coverage (primary region)')
ax_this.axhline(1, color='grey', lw=0.5)
ax_this.axvspan(-12.3, -11.75, color='grey', alpha=0.7, zorder=99)

ax_this.set_xlim(-20.25, -11.75)
Y0 = np.mean(X[p2_mask,0]-X[p2_mask,1], axis=0)
Yerr = np.percentile(X[p2_mask,0]-X[p2_mask,1], norm.sf([1, -1])*100, axis=0)

ax_second = plt.twinx(ax_this)
ax_second.bar(r, height=Y0, width=np.ediff1d(rbins), color="C4", alpha=0.3, label="No $z$; $\\mathcal{R}_\\mathrm{sat}\\sim0.005$")
ax_second.bar(r, height=np.mean(X[p2_mask,5], axis=0), width=np.ediff1d(rbins), color="C4", alpha=0.7, label="No $z$, dLSBG; $\\mathcal{R}_\\mathrm{sat}\\sim0.3$")

ax_second.errorbar(r, Y0, (Y0-Yerr[0], Yerr[1]-Y0), ls="", marker='', color="C4", alpha=0.6)

ax_second.set_ylim(0, 10.5)
ax_second.set_ylabel('# objects w/o redshift per host')

ax_this.legend(loc="lower left", bbox_to_anchor=(0, 0.25), fontsize=13, handlelength=1, title="For complete systems")
ax_second.legend(loc="lower left",  bbox_to_anchor=(0, 0), fontsize=13, handlelength=1)

fig.tight_layout()
plt.savefig("/home/yymao/Downloads/survey-completeness.pdf", bbox_inches="tight")

## calculate average R_sat

In [ ]:
basic = Query(C.r_abs_limit, ~C.has_spec, saga.host_catalog.construct_host_query("paper2_complete"))
sel1 = Query(basic, "human_selected == 0", C.paper2_targeting_cut)
sel2 = Query(basic, "human_selected > 0")

print(sel1.filter(base, "p_sat_corrected").mean())
print(sel2.filter(base, "p_sat_corrected").mean())